In [2]:
import numpy as np
import os
import glob
import pandas as pd

In [3]:
# --- 1. 경로 및 상수 설정 (로컬 환경용) ---
print("--- 1. 경로 설정 시작 ---")

# 현재 작업 디렉토리를 기준으로 경로 설정 (노트북 환경)
BASE_DIR = os.getcwd()

TOTAL_STEPS = 201             # 10ms (1 사이클) 동안의 총 파일 개수
NUM_CYCLES = 40               # 반복할 사이클 수

# Ex 및 Ez 파일이 들어있는 디렉토리 경로 (실제 폴더 이름 사용)
EX_DIR = os.path.join(BASE_DIR, "twin_cons_cathodicfirst_400us_100Hz_single10ms_Ex")
EZ_DIR = os.path.join(BASE_DIR, "twin_cons_cathodicfirst_400us_100Hz_single10ms_Ez")
OUTPUT_FILENAME = "E_field_40cycles.npy"
OUTPUT_PATH = os.path.join(BASE_DIR, OUTPUT_FILENAME)

print(f"\n--- 2. 설정된 경로 확인 ---")
print(f"기준 디렉토리: {BASE_DIR}")
print(f"Ex 경로: {EX_DIR}")
print(f"Ez 경로: {EZ_DIR}")
print(f"출력 파일: {OUTPUT_PATH}")

# 디렉토리 존재 확인
if not os.path.exists(EX_DIR):
    print(f"⚠️ 경고: Ex 디렉토리를 찾을 수 없습니다: {EX_DIR}")
if not os.path.exists(EZ_DIR):
    print(f"⚠️ 경고: Ez 디렉토리를 찾을 수 없습니다: {EZ_DIR}")

--- 1. 경로 설정 시작 ---

--- 2. 설정된 경로 확인 ---
기준 디렉토리: c:\Users\user\YongtaeC\neuron
Ex 경로: c:\Users\user\YongtaeC\neuron\twin_cons_cathodicfirst_400us_100Hz_single10ms_Ex
Ez 경로: c:\Users\user\YongtaeC\neuron\twin_cons_cathodicfirst_400us_100Hz_single10ms_Ez
출력 파일: c:\Users\user\YongtaeC\neuron\E_field_40cycles.npy


In [4]:
# Google Colab 관련 코드 제거됨 (로컬 환경용)

In [5]:
# --- 3. 데이터 로드 및 통합 함수 정의 (수정됨: r'' 사용) ---

def load_and_combine_e_field_data(component_dir, total_steps=201):
    """
    지정된 디렉토리의 모든 시간 스텝 파일을 로드하고 통합합니다.
    """

    file_pattern = os.path.join(component_dir, "*.txt")
    file_list = sorted(glob.glob(file_pattern))

    if len(file_list) != total_steps:
        if len(file_list) == 0:
             raise FileNotFoundError(f"'{component_dir}' 폴더에서 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        print(f"경고: 예상 파일 개수 ({total_steps}개)와 실제 파일 개수 ({len(file_list)}개)가 다릅니다.")

    try:
        # **수정된 부분: sep=r'\s+'**
        df_temp = pd.read_csv(file_list[0], skiprows=2, sep=r'\s+', header=None, engine='python')
        num_spatial_points = len(df_temp)
    except Exception as e:
        print(f"첫 번째 파일 로드 중 오류 발생 ({file_list[0]}): {e}")
        raise

    combined_data = np.zeros((num_spatial_points, total_steps), dtype=np.float32)

    print(f"\n-> 데이터 로드 시작. 디렉토리: {component_dir}, 공간 지점 수: {num_spatial_points}, 시간 스텝 수: {total_steps}")

    for i, file_path in enumerate(file_list):
        try:
            # **수정된 부분: sep=r'\s+'**
            df = pd.read_csv(file_path, skiprows=2, sep=r'\s+', header=None, engine='python')
            field_values = df.iloc[:, -1].values.astype(np.float32)

            if len(field_values) != num_spatial_points:
                raise ValueError(f"파일 {file_path}의 공간 지점 수가 일치하지 않습니다. (예상:{num_spatial_points}, 실제:{len(field_values)})")

            combined_data[:, i] = field_values

        except Exception as e:
            print(f"데이터 처리 중 오류 발생 (파일: {file_path}, 인덱스: {i}): {e}")
            combined_data[:, i] = 0

    return combined_data

In [6]:
# --- 4. 데이터 로드, 확장 및 저장 ---
try:
    print("\n--- 3. 데이터 로드 및 40 사이클 확장 ---")

    E_x_1cycle = load_and_combine_e_field_data(EX_DIR, total_steps=TOTAL_STEPS)
    E_z_1cycle = load_and_combine_e_field_data(EZ_DIR, total_steps=TOTAL_STEPS)

    # 데이터 확장
    E_x_40cycles = np.tile(E_x_1cycle, (1, NUM_CYCLES))
    E_z_40cycles = np.tile(E_z_1cycle, (1, NUM_CYCLES))

    final_time_steps = E_x_40cycles.shape[1]

    print(f"\n--- 4. 최종 데이터 확인 및 저장 ---")
    print(f"✅ 최종 Ex (40 사이클) shape: {E_x_40cycles.shape}")
    print(f"✅ 최종 Ez (40 사이클) shape: {E_z_40cycles.shape}")

    E_field_40cycles = np.stack((E_x_40cycles, E_z_40cycles), axis=0)

    print(f"✅ 최종 저장 배열 (2, Spatial, Time) shape: {E_field_40cycles.shape}")

    # .npy 파일을 로컬에 저장
    np.save(OUTPUT_PATH, E_field_40cycles)
    print(f"\n🎉 최종 데이터가 '{OUTPUT_PATH}' 파일로 성공적으로 저장되었습니다.")
    print(f"   파일 크기: {os.path.getsize(OUTPUT_PATH) / (1024**2):.2f} MB")

except FileNotFoundError as e:
    print(f"\n❌ 파일을 찾을 수 없습니다: {e}")
    print("디렉토리 경로와 파일 위치를 다시 한번 확인해 주세요.")
except Exception as e:
    print(f"\n❌ 처리 중 예상치 못한 오류 발생: {e}")
    import traceback
    traceback.print_exc()


--- 3. 데이터 로드 및 40 사이클 확장 ---

-> 데이터 로드 시작. 디렉토리: c:\Users\user\YongtaeC\neuron\twin_cons_cathodicfirst_400us_100Hz_single10ms_Ex, 공간 지점 수: 14641, 시간 스텝 수: 201

-> 데이터 로드 시작. 디렉토리: c:\Users\user\YongtaeC\neuron\twin_cons_cathodicfirst_400us_100Hz_single10ms_Ez, 공간 지점 수: 14641, 시간 스텝 수: 201

--- 4. 최종 데이터 확인 및 저장 ---
✅ 최종 Ex (40 사이클) shape: (14641, 8040)
✅ 최종 Ez (40 사이클) shape: (14641, 8040)
✅ 최종 저장 배열 (2, Spatial, Time) shape: (2, 14641, 8040)

🎉 최종 데이터가 'c:\Users\user\YongtaeC\neuron\E_field_40cycles.npy' 파일로 성공적으로 저장되었습니다.
   파일 크기: 898.08 MB
